## Feature Development Extraction: Neurophysiology [dyskinesia project]




<b> Content </b>


<b> Aperiodic estimates </b>
Relevant literature:
- Periodic and a-periodic components relevance and interaction, different reasons (per + a-per) for signal changes observed within a specific bandwidth. Aperiodic component (complicated) vs exponent (1/f) (Donoghue, ..., Shestyuk & Voytek, Nature Neurosc 2020 : https://www.nature.com/articles/s41593-020-00744-x)
- cycle-by-cycle features: bycycle toolbox (Cole & Voytek, J of Neurophys 2019, https://journals.physiology.org/doi/full/10.1152/jn.00273.2019)
- aperiodic component, PD severity, and cortico-subcortico-activity, Bush & Zou, Richardson, bioRxiv 2023 https://www.biorxiv.org/content/10.1101/2023.02.08.527719v1?rss=1

<b> Periodic component analysis: </b> 
- Try Wavelet Dceomposition vs Welch (tapered) Spectral Decomposition





### 0. Loading packages and functions, defining paths



In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import pandas as pd
import numpy as np
import sklearn as sk
from itertools import product, compress

import matplotlib.pyplot as plt
from  matplotlib import __version__ as plt_version



In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
# print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)
print('matplotlib', plt_version)
## FEB 2022:
# Python sys 3.9.7 (default, Sep 16 2021, 08:50:36) 
# [Clang 10.0.0 ]
# pandas 1.3.4
# numpy 1.20.3
# mne_bids 0.9
# mne 0.24.1
# sci-py 1.7.1
# sci-kit learn 1.0.1

In [ ]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [ ]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')
figpath = os.path.join(projectpath, 'figures')
datapath = os.path.join(projectpath, 'data')

In [ ]:
os.chdir(codepath)
# own utility functions
import utils.utils_fileManagement as utilsFiles
import utils.utils_windowing as utilsWindows
from utils.utils_fileManagement import (get_project_path,
                                        load_class_pickle,
                                        save_class_pickle,
                                        mergedData,
                                        correct_acc_class)

# own data exploration functions
import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_analysis.ft_processing_helpers as ftProc

import lfpecog_plotting.plotHelpers as plotHelp

### Plot ssd'd Spectral Descriptives

#### SSD time freq plotting


In [ ]:
import lfpecog_plotting.plot_timeFreqs_ssd_psds as plot_ssd_TFs

In [ ]:
WIN_LEN=10
WIN_OVERLAP=0.5
DATA_VERSION='v4.0'
FT_VERSION='v6'
SSD_BROAD=True
INCL_STN_ONLY_PTS=False
IGNORE_PTS=['011', '104', '106']

In [ ]:
importlib.reload(ftProc)
importlib.reload(importClin)
importlib.reload(plot_ssd_TFs)

# CONVERT ONLY STNS!!!!!
FIG_DATE = '0202b'
SUBS = utilsFiles.get_avail_ssd_subs(DATA_VERSION=DATA_VERSION,
                          FT_VERSION=FT_VERSION,
                          IGNORE_PTS=IGNORE_PTS)
print(SUBS)

for sub in SUBS:
    if sub not in ['008',]:
        print(f'skip {sub}')
        continue
    
    print(f'PLOT SUB-{sub}')
    plot_ssd_TFs.plot_indiv_ssd_timefreq_allSources(
        sub, fig_name_base=f'{FIG_DATE}_ssdTimeFreq',
        FT_VERSION=FT_VERSION,
        DATA_VERSION=DATA_VERSION,
        ZSCORE=False, LOG_POWER=True,
        SAVE_PLOT=True,
        FOR_FIG=True,)
    # except:
    #     print(f'sub {sub} error')

#### SSD PSD plotting

In [ ]:

import lfpecog_analysis.get_SSD_timefreqs as ssd_TimeFreq
# import lfpecog_analysis.process_connectivity as procConn
import lfpecog_analysis.get_acc_task_derivs as accDerivs
import lfpecog_plotting.plot_descriptive_SSD_PSDs as plot_PSDs
import lfpecog_plotting.plot_COH_spectra as plot_COHs
import lfpecog_analysis.psd_analysis_classes as psdAnlClass

In [ ]:
# WIN_LEN=10
# WIN_OVERLAP=0.5
DATA_VERSION='v4.0'
FT_VERSION='v6'
# SSD_BROAD=True
# INCL_STN_ONLY_PTS=True
IGNORE_PTS=['011', '104', '106']

In [ ]:
DATA_VERSION='v4.0'
FT_VERSION='v6'
# SSD_BROAD=True
# INCL_STN_ONLY_PTS=True
IGNORE_PTS=['011', '104', '106']

SUBS = utilsFiles.get_avail_ssd_subs(DATA_VERSION=DATA_VERSION,
                          FT_VERSION=FT_VERSION,
                          IGNORE_PTS=IGNORE_PTS)
print(SUBS)
print(len(SUBS))

In [ ]:
# importlib.reload(procConn)

importlib.reload(ssd_TimeFreq)


TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(
    SUBS=SUBS, FT_VERSION=FT_VERSION,
    DATA_VERSION=DATA_VERSION,
    GET_CONNECTIVITY=False,  # 'trgc'
)


In [ ]:
importlib.reload(psdAnlClass)
importlib.reload(ssd_TimeFreq)

COH_TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(
    SUBS=SUBS, COH_SOURCE='STNs',
    FEATURE='sqCOH',
    FT_VERSION=FT_VERSION,
    DATA_VERSION=DATA_VERSION,
    GET_CONNECTIVITY=False,  # 'trgc'
)

# COH_TF = ssd_TimeFreq.get_COH_timeFreq(sub='008', COH_TYPE='sqCOH', FT_VERSION='v6',)

Plot 10-second epochs using new (Feb-Mar 24) plotting scripts

In [ ]:
print(f"shape of .values array: {TFs['012']['lfp_left'].values.shape}")
print(f"shape of .freqs array: {TFs['012']['lfp_left'].freqs.shape}")
print(f"shape of .times array: {TFs['012']['lfp_left'].times.shape}")

In [ ]:
BLs = psdAnlClass.get_selectedEphys(
    FEATURE='POWER',
    STATE_SEL='baseline',
    MIN_SEL_LENGTH=10,
    LOAD_PICKLE=True,
    USE_EXT_HD=True,
    PREVENT_NEW_CREATION=False,
    RETURN_PSD_1sec=True,
    verbose=False,
)

In [ ]:
coh_BLs = psdAnlClass.get_selectedEphys(
    FEATURE='COH_STNs',
    COH_TYPE='sqCOH',
    STATE_SEL='baseline',
    MIN_SEL_LENGTH=10,
    LOAD_PICKLE=True,
    USE_EXT_HD=True,
    PREVENT_NEW_CREATION=False,
    RETURN_PSD_1sec=True,
    verbose=False,
)

In [ ]:
import lfpecog_features.feats_spectral_helpers as specHelp
import lfpecog_plotting.plot_psd_restvsmove as plotRestMovePsds
import lfpecog_plotting.plot_10s_epoch_psds as plot_10sEpochs

### Plot STN/ECoG Lateralisation during unilat. Dyskinesia

Shows lateralization of subthalamic and cortical changes during unilateral Dyskinesia epochs

In [ ]:
importlib.reload(specHelp)
importlib.reload(plotRestMovePsds)
importlib.reload(plot_10sEpochs)

BASE_METHOD = 'percChange'  # percChange / Z / no

### 10-sec PREP has to lead to psd_arrs, psd_freqs, psd_subs
plot_10sEpochs.plot_unilatLID_PSD_10s(
    TFs=TFs, BLs=BLs,
    BASE_METHOD=BASE_METHOD,
    FIG_DATE='0311',
    SMOOTH_WIN=2,
    PEAKSHIFT_GAMMA=False,
)


### Plot full 10sec PSDs per LID-category

Plots overall subthalamic and cortical powers and coherences, split over Dyskinesia severity (per 10-sec epochs)

In [ ]:
# Test extraction

# importlib.reload(plot_10sEpochs)


# (psd_arr,
#  ps_freqs,
#  sub_arr) = plot_10sEpochs.prep_10sPSDs_lidCategs(
#         TFs=COH_TFs, SOURCE='STNs',
#         BASELINE=coh_BLs,
#         FEATURE='sqCOH',
#         IPSI_CONTRA_UNILID=False,
#         BASE_METHOD='percChange',
#     )

In [ ]:
### Get Power Values

importlib.reload(plot_10sEpochs)

BASE_METHOD = 'percChange'  # percChange / Z / no

### 10-sec PREP has to lead to psd_arrs, psd_freqs, psd_subs
psd_arr_dict, ps_freqs_dict, sub_arr_dict = {}, {}, {}

for src in ['lfp', 'ecog']:

    (psd_arr_dict[src],
     ps_freqs_dict[src],
     sub_arr_dict[src]) = plot_10sEpochs.prep_10sPSDs_lidCategs(
        TFs=TFs, SOURCE=src,
        BASELINE=BLs,
        FEATURE='POWER',
        IPSI_CONTRA_UNILID=False,
        BASE_METHOD=BASE_METHOD,
    )


In [ ]:
importlib.reload(ssd_TimeFreq)
importlib.reload(plotRestMovePsds)
importlib.reload(plot_10sEpochs)

# uses pre calculated PSD values, COH values are calculated within function
plot_10sEpochs.plot_overall_PSD_COH(
    psd_arr_dict=psd_arr_dict, sub_arr_dict=sub_arr_dict,
    ps_freqs_dict=ps_freqs_dict,
    BASE_METHOD=BASE_METHOD,
    PEAKSHIFT_GAMMA=False,
    FIG_DATE='0313',
    SMOOTH_WIN=3,
)

### Plot Coherence Spectra Densities

Plot 4 STN-ECoG Coherencies

In [ ]:
importlib.reload(plot_COHs)

FIG_DATE = '1025'
CUT_MILD_MOD = 4
CUT_MOD_SEV = 8
INCL_CORE_CDRS = False
SAVE_PLOT = True
SHOW_PLOT = False
FILE_NAME = (f'{FIG_DATE}_STNECOG_4_COHs'
            f'_{CUT_MILD_MOD}mod{CUT_MOD_SEV}sev')
if not INCL_CORE_CDRS: FILE_NAME += '_noCore'

fig, axes = plt.subplots(2, 2, figsize=(12, 12),
                         sharey='row')

for i_row, COH_type in enumerate(['sq_coh', 'imag_coh']):
    
    for i_col, MOV in enumerate(['INCL_MOVE', 'EXCL_MOVE']):

        axes[i_row, i_col] = plot_COHs.plot_COH_spectra(
            COH_type=COH_type, COH_source='STN_ECOG',
            SELECT_ON_ACC_RMS=MOV,
            RETURN_AX=True,
            given_ax=axes[i_row, i_col],
            CUT_MILD_MOD=CUT_MILD_MOD,
            CUT_MOD_SEV=CUT_MOD_SEV,
            INCL_CORE_CDRS=INCL_CORE_CDRS,
        )

if SAVE_PLOT:
    path = os.path.join(get_project_path('figures'),
                'ft_exploration',
                f'data_{DATA_VERSION}_ft_{FT_VERSION}',
                'descr_COHs')
    if not os.path.exists(path): os.makedirs(path)
    plt.savefig(os.path.join(path, FILE_NAME),
                facecolor='w', dpi=300,)

if SHOW_PLOT: plt.show()
else: plt.close()

Plot 4 STN-STN Coherencies

In [ ]:
## PLOT STN-STN COHERENCES
importlib.reload(plot_COHs)

SAVE_PLOT = True
SHOW_PLOT = False
FILE_NAME = '1025_STNSTN_4_COHs'

fig, axes = plt.subplots(2, 2, figsize=(12, 12),
                         sharey='row')

for i_row, COH_type in enumerate(['sq_coh', 'imag_coh']):
    
    for i_col, MOV in enumerate(['INCL_MOVE', 'EXCL_MOVE']):

        axes[i_row, i_col] = plot_COHs.plot_COH_spectra(
            COH_type=COH_type, COH_source='STN_STN',
            SELECT_ON_ACC_RMS=MOV,
            RETURN_AX=True,
            given_ax=axes[i_row, i_col]
        )

if SAVE_PLOT:
    path = os.path.join(get_project_path('figures'),
                'ft_exploration',
                f'data_{DATA_VERSION}_ft_{FT_VERSION}',
                'descr_COHs')
    if not os.path.exists(path): os.makedirs(path)
    plt.savefig(os.path.join(path, FILE_NAME),
                facecolor='w', dpi=300,)

if SHOW_PLOT: plt.show()
else: plt.close()


##### Plot Group Results: Spectral changes over Time after L-Dopa, non-LID versus LID

Moved to separate py script (plot_group_metrics_over_time.py)

##### Plot LID-Group Results: Laterality of LID in STN and ECoG

Plot SEVERITY AND LATERALITY OF STN-changes during DYSKINESIA

- LAT_UNI: plot laterality of unilateral LID
- LAT_BILAT: plot laterality of only bilateral LID
- SCALE: combine all uni- and bilateral LID
    - PLOT_ONLY_MATCH == True: plot only STN contralateral to LID
    - PLOT_ONLY_MATCH == False: plot contra-, ipsi-, and bi-lateral STN-LID


In [ ]:
import lfpecog_analysis.psd_lid_stats as psd_stats
import lfpecog_analysis.get_acc_task_derivs as accDerivs

In [ ]:
importlib.reload(ftProc)
importlib.reload(accDerivs)
importlib.reload(importClin)
importlib.reload(plotHelp)
# importlib.reload(psd_stats)
importlib.reload(plot_PSDs)

# PM, needs: TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(SUBS=SUBS)

# 'LAT_UNI' , 'LAT_BILAT', 'SCALE', 'LAT_ALL_SCALE'
LAT_or_SCALE = 'LAT_UNI'
STAT_DATE = '1023'
FIG_DATE = '0305'

INCL_PRELID = False
PRELID_MINs=5
SMOOTH_FREQS=8
LOG_POWER=False
BASELINE_CORRECT=True
ZSCORE_FREQS=False
SHOW_GAMMA = False
SHOW_SIGN = False
PLOT_ONLY_MATCH = False
SELECT_ACC_RMS= False

fig_name = f'{FIG_DATE}_STN_PSDs_{LAT_or_SCALE}_LID_n{len(SUBS)}'

if INCL_PRELID: fig_name += f'_inclPre{PRELID_MINs}'
if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'
# if SINGLE_LINE: fig_name += '_lines'
if SHOW_GAMMA: fig_name += '_GAMMA'
if SHOW_SIGN: fig_name += '_SIGN'
if PLOT_ONLY_MATCH: fig_name += '_onlyMatch'
if SELECT_ACC_RMS: fig_name += f'_{SELECT_ACC_RMS}'


fsize=14

stn_stats = plot_PSDs.plot_STN_PSD_vs_LID(
    all_timefreqs=TFs,
    CDRS_RATER='Patricia',
    LAT_or_SCALE=LAT_or_SCALE,
    SELECT_ON_ACC_RMS=SELECT_ACC_RMS,
    incl_PRE_LID=INCL_PRELID,
    PRELID_MIN=PRELID_MINs,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=False,
    fsize=fsize,
    BREAK_X_AX=True,
    fig_name=fig_name,
    RETURN_FREQ_CORR=False,
    SHOW_ONLY_GAMMA=SHOW_GAMMA,
    SHOW_SIGN=SHOW_SIGN,
    PROCESS_STATS=False,
    p_SAVED_DATE=STAT_DATE,
    PLOT_ONLY_MATCH=PLOT_ONLY_MATCH,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)


Plot SEVERITY AND LATERALITY OF ECoG-changes during DYSKINESIA

In [ ]:
importlib.reload(accDerivs)
# importlib.reload(psdLID_stats)
importlib.reload(plot_PSDs)

# PM, needs: TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(SUBS=SUBS)

# 'LAT_UNI' , 'LAT_BILAT', 'SCALE', 'LAT_ALL_SCALE'
FIG_DATE = '1025'
STAT_DATE = '1023'
LAT_or_SCALE = 'SCALE'
SAVE_PLOT = True
SHOW_PLOT = False

SMOOTH_FREQS = 8
LOG_POWER = False
BASELINE_CORRECT = True
ZSCORE_FREQS = False
SINGLE_LINE = False
SHOW_GAMMA = False
PLOT_ONLY_MATCH = True
SHOW_SIGN = False
SELECT_ACC_RMS = 'EXCL_MOVE'

fig_name = f'{FIG_DATE}_ECoG_PSDs_LID_{LAT_or_SCALE}_n{len(SUBS)}'

if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'
if SINGLE_LINE: fig_name += '_lines'
if SHOW_GAMMA: fig_name += '_GAMMA'
if SHOW_SIGN: fig_name += '_SIGN'
if PLOT_ONLY_MATCH: fig_name += '_onlyMatch'
if SELECT_ACC_RMS: fig_name += f'_{SELECT_ACC_RMS}'

fsize=14

plot_PSDs.plot_ECOG_PSD_vs_LID(
    TFs,
    LAT_or_SCALE=LAT_or_SCALE,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SELECT_ON_ACC_RMS=SELECT_ACC_RMS,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=False,
    fsize=fsize,
    BREAK_X_AX=True,
    fig_name=fig_name,
    single_sub_lines=SINGLE_LINE,
    PLOT_ONLY_MATCH=PLOT_ONLY_MATCH,
    SHOW_ONLY_GAMMA=SHOW_GAMMA,
    SHOW_SIGN=SHOW_SIGN,
    PROCESS_STATS=False,
    p_SAVED_DATE=STAT_DATE,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)

Plot LATERALITY during UNILATERAL dyskinesia

In [ ]:
LID_SUBS = [s for s in SUBS if s not in ['101', '109', '017']]
print(LID_SUBS)

In [ ]:
importlib.reload(plot_PSDs)


SAVE_PLOT = True
SHOW_PLOT = False

SMOOTH_FREQS=8
LOG_POWER=False
BASELINE_CORRECT=True
ZSCORE_FREQS=False
CDRS_RATER = 'Patricia'

fig_name = f'0305_LATERALITY_STNandECOG_PSDs_unilatLID_n{len(SUBS)}'
if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'



fig, axes = plt.subplots(1, 2, figsize=(12, 6))

fsize=14

plot_PSDs.plot_STN_PSD_vs_LID(
    TFs, sel_subs=LID_SUBS,
    LAT_or_SCALE='LAT_UNI',  # LAT_UNI
    CDRS_RATER=CDRS_RATER,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=axes[0],
    fsize=fsize,
    BREAK_X_AX=True,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)
plot_PSDs.plot_ECOG_PSD_vs_LID(
    TFs, sel_subs=LID_SUBS,
    LAT_or_SCALE='LAT_UNI',
    CDRS_RATER=CDRS_RATER,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=axes[1],
    fsize=fsize,
    BREAK_X_AX=True,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)

# equalize axes
ymin = min([min(ax.get_ylim()) for ax in axes])
ymax = max([max(ax.get_ylim()) for ax in axes])
for ax in axes: ax.set_ylim(ymin, ymax)

for ax in axes: ax.tick_params(axis='both', size=fsize,
                               labelsize=fsize)
plt.tight_layout()

if SAVE_PLOT:
    plt.savefig(os.path.join(get_project_path('figures'),
                             'ft_exploration',
                             f'data_{DATA_VERSION}_ft_{FT_VERSION}',
                             'descr_PSDs', fig_name),
                facecolor='w', dpi=300,)
if SHOW_PLOT: plt.show()
else: plt.close()